Data Wrangling is done for the chennai state of India. size of data: 407,669,106bytes

The reason for selecting this part of the map is that I lived major part of my life in this city of Tamil Nadu India --Chennai.
The link to this map set https://mapzen.com/data/metro-extracts/metro/chennai_india/

Sites that i have been browsing regarding my Data wrangling:stackoverflow,Mongodb.com,http://wiki.openstreetmap.org/wiki/Accuracy.

In [ ]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import json
import codecs
# storing the path of the osm file in variable name-osm_file
osm_file='C:/Users/Praneetha/Documents/chennai_india.osm'
SAMPLE_FILE = "sample.osm"

k = 25 # Parameter: take every k-th top level element
# Parsing through the elements of the osm_file
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(osm_file)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')



Counting the tags in the osm_file:

In [30]:
def count_tags(SAMPLE_FILE):
    tags={}
    for elem in get_element(SAMPLE_FILE):
        if elem.tag in tags :
            tags[elem.tag] +=1
        else:
            tags[elem.tag] =1
            elem.clear()
            pprint.pprint(tags)
count_tags(osm_file)

{'node': 1}
{'node': 1826997, 'way': 1}
{'node': 1826997, 'relation': 1, 'way': 409883}


Getting the street types:

In [31]:
street_types_re=re.compile(r'\b\S+\.?$',re.IGNORECASE)
street_types=defaultdict(set)
expected=["Street","Avenue","Drive","Court","Place","Road","Nagar","Salai","Lane","Highway","South"]
mapping = { "St": "Street",
            "St.": "Street",
            "Ave":"Avenue",
            "Ave.":"Avenue",
            "Rd":"Road",
            "Rd.":"Road",
            "Ln":"Lane",
            "Extn.":"Extension",
            "Extn":"Extension",
            "Col":"Colony",
           "Hwy":"Highway",
           "sou":"South"
            }

def audit_street_type(street_types,street_name):
    m=street_types_re.search(street_name)
    if m:
        street_type=m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            return street_types

def is_street_name(elem):
    return (elem.attrib['k']=="addr:street")
def audit(SAMPLE_FILE):
    for elem in get_element(SAMPLE_FILE):
        if elem.tag=="way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types,tag.attrib['v'])
                    pprint.pprint(dict(street_types))
    return street_types
audit(SAMPLE_FILE)

{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave'])}
{'Ave': set(['Alumni Ave']), 'St': set(['S Mada St'])}
{'Ave': set(['Alumni Ave']),
 'St': set(['S Mada St']),
 'road': set(['Pallavaram - Duraipalkam 100 feet road'])}
{'Ave': set(['Alumni Ave']),
 'St': set(['S Mada St']),
 'road': set(['Pallavaram - Duraipalkam 100 feet road'])}
{'Ave': set(['Alumni Ave']),
 'St': set(['S Mada St']),
 'road': set(['Pallavaram - Duraipalkam 100 feet road']),
 'street': set(['Vadhyar Subbraya mudali street'])}
{'Ave': set(['Alumni Ave']),
 'Road)': set(['Anna Salai (Mount Road)']),
 'St': set(['S Mada St']),
 'road': set(['Pallavaram - Duraipalkam 100 feet road']),
 'street': set(['Vadhyar Subbraya mudali street'])}
{'Ave': set(['Alumni Ave']),
 'Road)': set(['Anna Salai (Mount Road)']),
 'St': set(['S Mada St']),
 'road': 

defaultdict(set,
            {'Ave': {'8th Ave', 'Alumni Ave'},
             'III': {'First Street, Maxworth Nagar - Phase III'},
             'Koyambedu': {'SAF Games Village, Koyambedu'},
             'Road)': {'Anna Salai (Mount Road)'},
             'St': {'S Mada St',
              'Shastri Nagar 1st Cross St',
              'Tailor Estate 2nd St'},
             'Thuraipakkam': {'Fourrts Avenue, Thuraipakkam'},
             'nagar': {'7th Cross Street, Rajalaksmi nagar'},
             'post': {'Anna Main Road, Kolapakkam, Gerugambakkam post'},
             'road': {'2nd Circular road',
              'Pallavaram - Duraipalkam 100 feet road'},
             'road)': {'Maharajapuram Santhanam Salai (Griffith road)'},
             'street': {'Brahmin street', 'Vadhyar Subbraya mudali street'},
             'street)': {'big street(veera ragavan street)'}})

Problems encountered:The street names have to be corrected as not all streets are marked as 'street' few are stated as 'st'.updating similar changes to the dataset is done below:
Updating the street names

In [34]:
def update_name(name, mapping):

    m=street_types_re.search(name)
    if m:
        if m.group() in mapping.keys():
           name= re.sub(m.group(),mapping[m.group()],name)
    return name

def test():
 st_types=audit(SAMPLE_FILE)
 for st_type,ways in street_types.iteritems():
       for name in ways:
           better_name = update_name(name, mapping)
#            print better_name
test()


{'Arumbakkam': set(['N.S.K. Nagar 1st Street, N.S.K Nagar, Arumbakkam']),
 'Ave': set(['8th Ave', 'Alumni Ave', 'Hostel Ave']),
 'Broadway)': set(['Prakasam Road (Broadway)']),
 'Chemencheri': set(['Chemencheri']),
 'Chennai': set(['Balakrishnan Street, Nanmangalam, Chennai']),
 'Coimbet': set(['Coimbet']),
 'Colony': set(['Nathan Subramaniam Colony']),
 'East': set(['Ambattur Industrial Estate Road, Mogappair, T S Krishna Nagar, J Jayalalitha Nagar, Mogappair East',
              'Velammal Gardens, T S Krishna Nagar, Mogappair East']),
 'Extension': set(['School Rd,Anna Nagar West Extension']),
 'Extn': set(['6th Main Road Ram Nagar (N) Extn']),
 'Extn.': set(['Umapathy street Extn.']),
 'Guduvanchery,': set(['NH Service Road, Guduvanchery,']),
 'Hasthinapuram': set(['Srinivasa Nagar, Hasthinapuram']),
 'II': set(['II Avenue Maxworth Nagar Phase II']),
 'III': set(['First Street, Maxworth Nagar - Phase III',
             'Fourth Street, Maxworth Nagar - Phase III',
             'Secon

Converting into json format so that it can be imported into MongoDb for further analysis:

In [60]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = {'version': None, 'timestamp': None, 'changeset': None, 'user': None,
           'uid': None}
def shape_element(element):
    node = {}
    address = {}
    amenity={}
    religion={}
    cuisine={}
    pos = []

    if element.tag == "node" or element.tag == "way" :
        node["id"] = element.attrib["id"]
        node["type"] =  element.tag
        node[ "visible"] = element.get("visible")
        created = {}
        created["version"] = element.attrib["version"]
        created["changeset"] = element.attrib["changeset"]
        created["timestamp"] = element.attrib["timestamp"]
        created["user"] = element.attrib["user"]
        node["created"] = created

        if "lat" in element.keys() and "lon" in element.keys():
           pos = [element.attrib["lat"], element.attrib["lon"]]
           node["pos"] = [float(string) for string in pos]

        else:
           node["pos"] = None

        for tag in element.iter('tag'):
           if re.search('amenity',tag.attrib['k']):
                a=tag.attrib['k']
                amenity[a]=tag.attrib['v']
                node['amenity']=amenity
           if re.search('religion',tag.attrib['k']):
                r=tag.attrib['k']
                religion[r]=tag.attrib['v']
                node['religion']=religion 
           if re.search('cuisine',tag.attrib['k']):
                c=tag.attrib['k']
                cuisine[c]=tag.attrib['v']
                node['cuisine']=cuisine     
           if re.search('addr:', tag.attrib['k']):
                if len(tag.attrib['k'].split(":")) < 3:
                    addr_add = tag.attrib['k'].split(":")[1]
                    address[addr_add] = tag.attrib['v']

        if address:
            node['address'] = address
        for nd in element.iter("nd"):
            if not "node_refs" in node.keys():
                node["node_refs"] = []
            node_refs = node["node_refs"]
            node_refs.append(nd.attrib["ref"])
            node["node_refs"] = node_refs
        return node
    else:
        return None

In [61]:
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for element in get_element(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data
data = process_map(SAMPLE_FILE, True)
# pprint.pprint(data)

Connecting to MongoDB and executing queries based on the imported .json file:

In [76]:
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    pipeline = [{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$sort":{"count":-1}},{"$limit":17}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.osmdata.aggregate(pipeline)]
# Top 1 contributing user
db = get_db('examples')
pipeline = make_pipeline()
result = aggregate(db, pipeline)
print result

[{u'count': 16637, u'_id': u'maheshrkm'}, {u'count': 10601, u'_id': u'PlaneMad'}, {u'count': 9330, u'_id': u'venkatkotha'}, {u'count': 5558, u'_id': u'shekarn'}, {u'count': 5520, u'_id': u'praveeng'}, {u'count': 5391, u'_id': u'sramesh'}, {u'count': 5302, u'_id': u'Rahuldhanraj'}, {u'count': 5266, u'_id': u'anthony1'}, {u'count': 5248, u'_id': u'kushwanth'}, {u'count': 4938, u'_id': u'venkatesh10'}, {u'count': 4791, u'_id': u'rajureddyvudem'}, {u'count': 4708, u'_id': u'saikumar'}, {u'count': 4632, u'_id': u'ravikumar1'}, {u'count': 4562, u'_id': u'harishvarma'}, {u'count': 4445, u'_id': u'vamshiN'}, {u'count': 4192, u'_id': u'shiva05'}, {u'count': 4135, u'_id': u'Apreethi'}]


In [74]:
# Number of users appearing only once (having 1 post)
u=db.osmdata.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$group":{"_id":"$count", "num_users":{"$sum":1}}}, {"$sort":{"_id":1}}, {"$limit":1}])
for doc in u:
    print doc

{u'num_users': 182, u'_id': 1}


Additional data exploration using MongoDb queries:
Counting the number of nodes and ways using MongoDb:
Counting the number of unique users:    

In [72]:
# Total documents
doc_count=db.osmdata.find().count()
# node count
node_count=db.osmdata.find({"type":"node"}).count()
# unique users
unique_user=len(db.osmdata.distinct("created.user"))
# way count
way_count=db.osmdata.find({"type":"way"}).count()
print node_count
print unique_user
print way_count
print doc_count

182700
536
40988
223688


In [73]:
# Top 10 amenities
a=db.osmdata.aggregate([{"$match":{"amenity":{"$exists":1}}},
                        {"$group":{"_id":"$amenity","count":{"$sum":1}}}, 
                        {"$sort":{"count":-1}}, {"$limit":10}])
for doc in a:
   print doc

{u'count': 49, u'_id': {u'amenity': u'place_of_worship'}}
{u'count': 42, u'_id': {u'amenity': u'school'}}
{u'count': 27, u'_id': {u'amenity': u'restaurant'}}
{u'count': 24, u'_id': {u'amenity': u'hospital'}}
{u'count': 23, u'_id': {u'amenity': u'atm'}}
{u'count': 16, u'_id': {u'amenity': u'fuel'}}
{u'count': 15, u'_id': {u'amenity': u'college'}}
{u'count': 13, u'_id': {u'amenity': u'bank'}}
{u'count': 11, u'_id': {u'amenity': u'bus_station'}}
{u'count': 10, u'_id': {u'amenity': u'pharmacy'}}


From the above results we can see that top amenity is the Place of worship naturally as chennai is called the temple town.

In [65]:
# Top religion
r=db.osmdata.aggregate([{"$match":{"amenity":{"$exists":1}}},                                              
{"$group":{"_id":"$religion", "count":{"$sum":1}}},                                                
{"$sort":{"count":-1}}])
for doc in r:
   print doc

{u'count': 306, u'_id': None}
{u'count': 30, u'_id': {u'religion': u'hindu'}}
{u'count': 7, u'_id': {u'religion': u'christian'}}
{u'count': 2, u'_id': {u'religion': u'muslim'}}
{u'count': 1, u'_id': {u'religion': u'unitarian'}}


Though the first highest is None we can see that there is some information missing but taking account of other results we can say that Hindu religion is high in number.

In [70]:
# Top cuisine
c=db.osmdata.aggregate([{"$match":{"amenity":{"$exists":1}}}, 
                        {"$group":{"_id":"$cuisine", "count":{"$sum":1}}}, 
                        {"$sort":{"count":-1}}])
for doc in c:
    print doc


{u'count': 328, u'_id': None}
{u'count': 5, u'_id': {u'cuisine': u'indian'}}
{u'count': 3, u'_id': {u'cuisine': u'chinese'}}
{u'count': 2, u'_id': {u'cuisine': u'pizza'}}
{u'count': 2, u'_id': {u'cuisine': u'regional'}}
{u'count': 2, u'_id': {u'cuisine': u'ice_cream'}}
{u'count': 1, u'_id': {u'cuisine': u'juice'}}
{u'count': 1, u'_id': {u'cuisine': u'burger'}}
{u'count': 1, u'_id': {u'cuisine': u'korean'}}
{u'count': 1, u'_id': {u'cuisine': u'South_Indian'}}


Top cuisine of chennai is the Indian cuisine.

Overview of data:
More clearity in the data is required as we can see the top count in cuisine and religion is None.Further information provided will give us a clear idea regarding the result.First 10 Top contributing users seems to be very high compared to other users.

Further Improvements:
Naming is not always unambiguous and there are many sources of confusion - from different governmental entities naming the features differently, to inaccurate signposting.
OpenStreetMap tries to map reality 'on the ground'.
What do the street signs say? What do the local inhabitants call the place?
Benefits:
I think that local names can also be added to the map would help the tourists and people new to that particular city.
Problems:
The data may look cluttered on the map.